In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Charger les données

In [2]:
def load_by_chunks(path: str, chunk_size: int = 50_000) -> pd.DataFrame:
    chunks = pd.read_csv(path, chunksize=chunk_size)
    return pd.concat(chunks, ignore_index=True)

X_test = load_by_chunks('X_test_clean.csv')
X_train = load_by_chunks('X_train_clean.csv') 

print("Noms des colonnes :", X_test.columns.tolist())

# Affichage des dimensions et d'un aperçu des données pour vérification
print("X_test shape:", X_test.shape)
print(X_test.head())
print("\nX_train shape:", X_train.shape)
print(X_train.head())

/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns (57,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.concat(chunks, ignore_index=True)
/tmp/ipykernel_116762/3470133595.py:3: DtypeWarning: Columns

Noms des colonnes : ['Unnamed: 0', 'row_index', 'piezo_station_department_code', 'piezo_station_update_date', 'piezo_station_investigation_depth', 'piezo_station_department_name', 'piezo_station_commune_code_insee', 'piezo_station_pe_label', 'piezo_station_bdlisa_codes', 'piezo_station_altitude', 'piezo_station_bss_code', 'piezo_station_commune_name', 'piezo_station_longitude', 'piezo_station_latitude', 'piezo_station_bss_id', 'piezo_bss_code', 'piezo_measurement_date', 'piezo_obtention_mode', 'piezo_status', 'piezo_qualification', 'piezo_continuity_code', 'piezo_continuity_name', 'piezo_producer_code', 'piezo_producer_name', 'piezo_measure_nature_code', 'piezo_measure_nature_name', 'meteo_id', 'meteo_name', 'meteo_latitude', 'meteo_longitude', 'meteo_altitude', 'meteo_date', 'meteo_rain_height', 'meteo_temperature_min', 'meteo_time_tn', 'meteo_temperature_max', 'meteo_time_tx', 'meteo_temperature_avg', 'meteo_temperature_avg_threshold', 'meteo_frost_duration', 'meteo_amplitude_tn_tx',

### Extraction des features de BDLISA

In [3]:
from sklearn.preprocessing import LabelEncoder

BDLISA_FEATURES = ['EtatEH', 'NatureEH', 'MilieuEH', 'ThemeEH', 'OrigineEH']

def convert_bdlisa_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renvoie un nouveau dataframe en remplaçant le code BDLISA par 5 nouvelles colonnes
    correspondant aux caractéristiques de la zone.
    """
    # Les caractéristiques associées à la zone
    features = BDLISA_FEATURES

    # Téléchargé via https://reseau.eaufrance.fr/geotraitements/bdlisa/files/telechargement/BDLISA_V3/BDLISA_V3_NATIONAL-csv.zip
    # Il faut bien convertir `BDLISA_V3_NATIONAL/TME.csv` avec fin de ligne LF
    tme = pd.read_csv('BDLISA_V3_NATIONAL/TME.csv')
    tme = (tme[['CodeEH'] + features]
        .replace('X', -1) # La valeur -1 n'est pas prise
        .astype({f: 'int' for f in features}, errors='ignore')
    )
    
    for feature in features:
        # Ré-encoder les catégories sous la forme 0, 1, ..., N - 1
        label_encoded = LabelEncoder().fit_transform(tme[feature])
        tme[feature] = label_encoded
    
    df = df.dropna(subset=['piezo_station_bdlisa_codes'])
    df['CodeEH'] = df['piezo_station_bdlisa_codes'].apply(
        # un code BDLISA est de la forme "['516AA00']" ou "['516AA00', '516AF00']" par exemple.
        # On ne prend que le premier code de la liste.
        lambda s: s.split("'")[1]
    )
    return df.merge(tme, on='CodeEH').drop(columns=['CodeEH', 'piezo_station_bdlisa_codes'])

In [4]:
X_train_preprocessed = convert_bdlisa_codes(X_train)
X_test_preprocessed = convert_bdlisa_codes(X_test)

In [5]:
X_train_preprocessed[BDLISA_FEATURES].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2531338 entries, 0 to 2531337
Data columns (total 5 columns):
 #   Column     Dtype
---  ------     -----
 0   EtatEH     int64
 1   NatureEH   int64
 2   MilieuEH   int64
 3   ThemeEH    int64
 4   OrigineEH  int64
dtypes: int64(5)
memory usage: 96.6 MB


In [6]:
X_train_preprocessed.to_csv('X_train_preprocessed.csv')
X_test_preprocessed.to_csv('X_test_preprocessed.csv')